In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# use OpenAi client python library to call Ollama


In [ ]:
load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")

if openai_key:
    print(f"OpenAI API Key exists and begins {openai_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_key:
    print(f"Google API Key exists and begins {google_key[:8]}")
else:
    print("Google API Key not set")


In [ ]:
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant that responds in Markdown"
user_prompt = "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

In [ ]:


OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=prompts
)

print(response.choices[0].message.content)

In [ ]:
gpt_model = "gpt-4.1-mini"
olamma_model = "llama3.2"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

olamma_system_system = "You are a angry, courteous chatbot. You try not! to agree with \
everything the other person says, or find common ground."

gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": ollama})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_ollama():
    messages = []
    for gpt_msg, ollama_msg in zip(gpt_messages, ollama_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "assistant", "content": ollama_msg})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    message = ollama_via_openai.chat.completions.create(
        model=olamma_model,  # Se till att du har rätt modell här
        messages=messages
    )

    # Här hämtar vi rätt del av svaret, använd .choices och inte subscripts
    if message.choices:
        return message.choices[0].message.content
    else:
        return "No content returned"

In [ ]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")
print(f"GPT Messages: {gpt_messages}")
print(f"Ollama Messages: {ollama_messages}")

# Loopa och skapa nästa svar från både GPT och Ollama
for i in range(5):
    gpt_next = call_gpt()  # Få svar från GPT
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)  # Lägg till GPT:s svar till listan
    
    claude_next = call_ollama()  # Få svar från Ollama
    print(f"Ollama:\n{claude_next}\n")
    ollama_messages.append(claude_next)  # Lägg till Ollamas svar till listan